# FAISS Embeddingek Kiértékelése - CourtRankRL Projekt

Ez a notebook a google/embeddinggemma-300m modellel generált FAISS indexben tárolt embeddingeket elemzi. Az agents.md specifikáció alapján készített kiértékelési szempontokat vizsgálja a jelenlegi adatokkal.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import json
import faiss
from pathlib import Path
from typing import List, Dict, Any

# Plot stílus beállítása
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
from configs import config

print("CourtRankRL - FAISS Embedding Analysis")
print(f"Embedding model: {config.QWEN3_MODEL_NAME}")
print(f"Embedding dimension: {config.EMBEDDING_DIMENSION}")

## 1. FAISS Index és Embeddingek Betöltése

A jelenlegi projektben található FAISS index és chunk ID mapping betöltése.

In [ ]:
# FAISS index betöltése
faiss_path = config.FAISS_INDEX_PATH
chunk_map_path = config.CHUNK_ID_MAP_PATH

print(f"FAISS index betöltése: {faiss_path}")
print(f"Chunk ID mapping betöltése: {chunk_map_path}")

index = None
chunk_id_map = None
embeddings = None

try:
    if faiss_path.exists():
        index = faiss.read_index(str(faiss_path))
        print(f"✅ FAISS index betöltve: {index.ntotal} vektor, {index.d} dimenzió")
        
        # Embedding dimenzió ellenőrzése
        if hasattr(index, 'd') and index.d != config.EMBEDDING_DIMENSION:
            print(f"⚠️  Embedding dimenzió eltérés: index={index.d}, config={config.EMBEDDING_DIMENSION}")
        
        # Embeddingek kivonása (minden vektor)
        embeddings = []
        for i in range(index.ntotal):
            embedding = index.reconstruct(i)
            embeddings.append(embedding)
        embeddings = np.array(embeddings)
        print(f"✅ Embeddingek kivonva: {embeddings.shape}")
    else:
        print(f"❌ FAISS index nem található: {faiss_path}")
        print("Futtassa a gemma_embedding_runpod.ipynb-t először!")
except Exception as e:
    print(f"❌ Hiba a FAISS index betöltése során: {e}")
    index = None

# Chunk ID mapping betöltése
try:
    if chunk_map_path.exists():
        with open(chunk_map_path, 'r', encoding='utf-8') as f:
            chunk_id_map = json.load(f)
        print(f"✅ Chunk ID map betöltve: {len(chunk_id_map)} mapping")
    else:
        print(f"❌ Chunk ID map nem található: {chunk_map_path}")
except Exception as e:
    print(f"❌ Hiba a chunk ID map betöltése során: {e}")
    chunk_id_map = None

## 2. Chunk Adatok Betöltése

A chunks.jsonl fájl betöltése, hogy összekapcsoljuk az embeddingeket a szövegekkel és metadatokkal.

In [ ]:
# Chunkok betöltése mintavételezéssel a teljesítmény érdekében
chunks_file = config.CHUNKS_JSONL
sample_size = min(10000, index.ntotal if index else 0)  # Maximum 10k chunk elemzésre

df = None

if chunks_file.exists() and index is not None and chunk_id_map is not None:
    try:
        print(f"📊 Mintavétel: {sample_size} chunk betöltése...")
        
        chunks_list = []
        chunk_ids = list(chunk_id_map.values())[:sample_size]
        
        with open(chunks_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    chunk = json.loads(line.strip())
                    if chunk.get('chunk_id') in chunk_ids:
                        chunks_list.append(chunk)
                except json.JSONDecodeError:
                    continue
                if len(chunks_list) >= sample_size:
                    break
        
        if chunks_list:
            df = pd.DataFrame(chunks_list)
            print(f"✅ Betöltött chunkok száma: {len(df)}")
            
            # Embeddingek hozzárendelése a chunkokhoz
            if embeddings is not None:
                try:
                    embedding_dict = {chunk_id_map[str(i)]: embeddings[i] for i in range(len(embeddings))}
                    df = df.assign(embedding=df['chunk_id'].map(embedding_dict))
                    valid_embeddings = df['embedding'].notna().sum()
                    print(f"✅ Embeddingek hozzárendelve: {valid_embeddings} chunk")
                except Exception as e:
                    print(f"❌ Hiba az embeddingek hozzárendelése során: {e}")
            else:
                print("⚠️ Nincsenek embeddingek a hozzárendeléshez")
        else:
            print("⚠️ Nem találhatóak megfelelő chunkok")
    
    except Exception as e:
        print(f"❌ Hiba a chunkok betöltése során: {e}")
        df = None
else:
    print("⚠️ Hiányzó adatok a chunk betöltéshez:")
    if not chunks_file.exists():
        print(f"  - Chunks fájl nem található: {chunks_file}")
    if index is None:
        print("  - FAISS index")
    if chunk_id_map is None:
        print("  - Chunk ID map")
    df = None

# Ellenőrzés
if df is not None and not df.empty:
    print(f"\nAdatok betöltve: {df.shape[0]} chunk, {df.shape[1]} oszlop")
    print(f"Oszlopok: {df.columns.tolist()}")
else:
    print("\n❌ Nincs adat az elemzéshez")

## 3. Embedding Minőség Ellenőrzése

Az agents.md specifikáció szerint L2-normalizált embeddingek szükségesek a FAISS IP metrikához.

In [ ]:
if df is not None and 'embedding' in df.columns and embeddings is not None:
    print("🔍 Embeddingek minőségi ellenőrzése:")
    
    # Embedding dimenzió ellenőrzése
    first_embedding = df['embedding'].iloc[0]
    if isinstance(first_embedding, np.ndarray):
        print(f"✅ Embedding típusa: {type(first_embedding)}")
        print(f"✅ Embedding dimenziója: {len(first_embedding)}")
        print(f"✅ Elvárt dimenzió: {config.EMBEDDING_DIMENSION}")
        
        # L2 normalizálás ellenőrzése
        norms = df['embedding'].apply(lambda x: np.linalg.norm(x) if isinstance(x, np.ndarray) else 1.0)
        print(f"\nNormák statisztikái:")
        print(norms.describe())
        
        # Normalizálás ellenőrzése (agents.md spec szerint kötelező)
        normalized_count = norms.apply(lambda x: abs(x - 1.0) < 0.01).sum()
        print(f"L2-normalizált embeddingek: {normalized_count}/{len(df)} ({100*normalized_count/len(df):.1f}%)")
        
        if normalized_count < len(df):
            print("⚠️ Nem minden embedding van L2-normalizálva - ez problémás lehet FAISS IP metrikánál")
        
        # Hiányzó embeddingek
        missing_embeddings = df['embedding'].isna().sum()
        print(f"Hiányzó embeddingek: {missing_embeddings}")
        
        # Embeddingek közötti távolságok elemzése
        valid_embeddings = df.dropna(subset=['embedding'])
        if len(valid_embeddings) > 100:
            X = np.vstack(valid_embeddings['embedding'].values)
            
            # Véletlenszerűen kiválasztott 1000 pár távolsága
            n_pairs = min(1000, len(X) * (len(X) - 1) // 2)
            indices = np.random.choice(len(X), size=min(len(X), 100), replace=False)
            
            distances = []
            for i in range(len(indices)):
                for j in range(i + 1, len(indices)):
                    dist = np.linalg.norm(X[indices[i]] - X[indices[j]])
                    distances.append(dist)
            
            distances = np.array(distances)
            print(f"\nEmbeddingek közötti átlagos távolság: {distances.mean():.4f} ± {distances.std():.4f}")
            print(f"Távolság tartomány: [{distances.min():.4f}, {distances.max():.4f}]")
            
            # Embedding sűrűség
            print(f"Embedding dimenzió: {X.shape[1]}")
            print(f"Adatpontok száma: {len(X)}")
            print(f"Adatpontok sűrűsége: {len(X) / X.shape[1]:.2f}")
        else:
            print(f"⚠️ Túl kevés embedding a minőségi metrikákhoz: {len(valid_embeddings)}")
    else:
        print(f"❌ Embedding típusa nem megfelelő: {type(first_embedding)}")
else:
    print("❌ Nincs embedding adat az elemzéshez")

## 4. PCA Dimenziócsökkentés és Vizualizáció

Az embeddingek 2D-s leképezése PCA segítségével.

In [ ]:
if df is not None and 'embedding' in df.columns and embeddings is not None:
    # Hiányzó embeddingek eltávolítása
    valid_df = df.dropna(subset=['embedding']).copy()
    
    if len(valid_df) > 100:  # Minimum 100 embedding PCA-hoz
        print(f"📊 PCA elemzés {len(valid_df)} embeddinggel...")
        
        # Embeddingek NumPy tömbbé alakítása
        X = np.vstack(valid_df['embedding'].values)
        print(f"PCA bemenet: {X.shape}")
        
        # PCA futtatása
        pca = PCA(n_components=2)
        X_reduced = pca.fit_transform(X)
        
        print(f"PCA első két komponens varianciája: {pca.explained_variance_ratio_}")
        print(f"Összes magyarázott variancia: {sum(pca.explained_variance_ratio_):.3f}")
        
        # PCA eredmény vizualizáció
        plt.figure(figsize=(10, 7))
        plt.scatter(X_reduced[:, 0], X_reduced[:, 1], s=2, alpha=0.6)
        plt.title("EmbeddingGemma-300m - PCA 2D leképezés")
        plt.xlabel("Főkomponens 1")
        plt.ylabel("Főkomponens 2")
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Jogterület szerinti színezés
        if 'JogTerulet' in valid_df.columns:
            plt.figure(figsize=(12, 8))
            domains = valid_df['JogTerulet'].fillna('ismeretlen').values
            scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                                c=valid_df['JogTerulet'].astype('category').cat.codes, 
                                s=8, alpha=0.6, cmap='tab10')
            plt.title("EmbeddingGemma Embeddingek jogterület szerint színezve")
            plt.xlabel("Főkomponens 1")
            plt.ylabel("Főkomponens 2")
            plt.legend(handles=scatter.legend_elements()[0], 
                      labels=valid_df['JogTerulet'].astype('category').cat.categories.tolist(), 
                      bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.grid(True, alpha=0.3)
            plt.show()
        
        # Bíróság szerinti színezés
        if 'birosag' in valid_df.columns:
            plt.figure(figsize=(12, 8))
            courts = valid_df['birosag'].fillna('ismeretlen').values
            scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                                c=valid_df['birosag'].astype('category').cat.codes, 
                                s=8, alpha=0.6, cmap='Set3')
            plt.title("EmbeddingGemma Embeddingek bíróság szerint színezve")
            plt.xlabel("Főkomponens 1")
            plt.ylabel("Főkomponens 2")
            plt.legend(handles=scatter.legend_elements()[0], 
                      labels=valid_df['birosag'].astype('category').cat.categories.tolist(), 
                      bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.grid(True, alpha=0.3)
            plt.show()
        
        # Embedding hossza szerinti színezés
        if 'karakter_szam' in valid_df.columns:
            plt.figure(figsize=(12, 8))
            scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], 
                                c=valid_df['karakter_szam'], s=8, alpha=0.6, cmap='viridis')
            plt.title("EmbeddingGemma Embeddingek szöveghossz szerint színezve")
            plt.xlabel("Főkomponens 1")
            plt.ylabel("Főkomponens 2")
            plt.colorbar(scatter, label='Karakterek száma')
            plt.grid(True, alpha=0.3)
            plt.show()
    else:
        print(f"⚠️ Túl kevés embedding PCA-hoz: {len(valid_df)}")
else:
    print("❌ Nincs embedding adat a PCA-hoz")

## 5. Metadatok és Embeddingek Kapcsolata

Az embeddingek és a jogi metadatok közötti kapcsolat elemzése.

In [ ]:
if df is not None and 'embedding' in df.columns and not df.empty:
    valid_df = df.dropna(subset=['embedding'])
    
    if len(valid_df) > 0:
        print("📊 Metadatok és embeddingek kapcsolata:")
        
        # Bíróság megoszlás
        if 'birosag' in valid_df.columns:
            print(f"\nBíróságok megoszlása:")
            court_counts = valid_df['birosag'].value_counts()
            print(f"Top 10 bíróság: {court_counts.head(10).to_dict()}")
            
            plt.figure(figsize=(10, 6))
            court_counts.head(10).plot(kind='bar')
            plt.title('Top 10 leggyakoribb bíróság az embeddingekkel rendelkező chunkokban')
            plt.xlabel('Bíróság')
            plt.ylabel('Chunkok száma')
            plt.xticks(rotation=45, ha='right')
            plt.grid(axis='y')
            plt.show()
        
        # Jogterület megoszlás
        if 'JogTerulet' in valid_df.columns:
            print(f"\nJogterületek megoszlása:")
            domain_counts = valid_df['JogTerulet'].value_counts()
            print(f"Top 10 jogterület: {domain_counts.head(10).to_dict()}")
            
            plt.figure(figsize=(10, 6))
            domain_counts.head(10).plot(kind='bar')
            plt.title('Top 10 leggyakoribb jogterület az embeddingekkel rendelkező chunkokban')
            plt.xlabel('Jogterület')
            plt.ylabel('Chunkok száma')
            plt.xticks(rotation=45, ha='right')
            plt.grid(axis='y')
            plt.show()
        
        # Év szerinti megoszlás
        if 'HatarozatEve' in valid_df.columns:
            print(f"\nHatározatok év szerinti megoszlása:")
            year_counts = valid_df['HatarozatEve'].value_counts().sort_index()
            print(f"Évek tartomány: {year_counts.index.min()} - {year_counts.index.max()}")
            
            plt.figure(figsize=(12, 6))
            year_counts.plot(kind='line', marker='o')
            plt.title('Határozatok eloszlása év szerint')
            plt.xlabel('Év')
            plt.ylabel('Chunkok száma')
            plt.grid(True)
            plt.show()
        
        # Szöveghossz eloszlás
        if 'text' in valid_df.columns:
            valid_df['text_length'] = valid_df['text'].astype(str).apply(len)
            print(f"\nSzöveghossz statisztikák:")
            print(valid_df['text_length'].describe())
            
            plt.figure(figsize=(10, 6))
            plt.hist(valid_df['text_length'], bins=50, alpha=0.7)
            plt.title('Chunk szöveghossz eloszlása')
            plt.xlabel('Karakterek száma')
            plt.ylabel('Chunkok száma')
            plt.grid(True)
            plt.show()
    else:
        print("⚠️ Nincs érvényes embedding adat a metadatok elemzéséhez")
else:
    print("❌ Nincs adat a metadatok elemzéséhez")

## 6. FAISS Index Tulajdonságok

Az index típusának és konfigurációjának elemzése.

In [ ]:
if index is not None:
    print("🔍 FAISS Index tulajdonságok:")
    
    print(f"Index típusa: {type(index).__name__}")
    print(f"Vektorok száma: {index.ntotal}")
    print(f"Dimenzió: {index.d}")
    
    # Index specifikus tulajdonságok
    if hasattr(index, 'nlist'):
        print(f"IVF lista szám: {index.nlist}")
    if hasattr(index, 'nprobe'):
        print(f"Keresési próbák: {index.nprobe}")
    if hasattr(index, 'metric_type'):
        print(f"Metrika típusa: {index.metric_type}")
    
    # Index teljesítmény metrikák
    print(f"\nIndex mérete (becsült): {index.ntotal * index.d * 4 / (1024**2):.1f} MB")
    
    # Keresési sebesség becslés (ha van adat)
    if embeddings is not None and len(embeddings) > 0:
        # Egyszerű keresési teszt
        query_embedding = embeddings[0].reshape(1, -1)
        k = 10
        
        import time
        start_time = time.time()
        distances, indices = index.search(query_embedding.astype(np.float32), k)
        search_time = time.time() - start_time
        
        print(f"Keresési teljesítmény (1 query, top-{k}): {search_time*1000:.2f}ms")
        print(f"Átlagos távolság: {distances[0].mean():.4f}")
        print(f"Távolság szórás: {distances[0].std():.4f}")
    
    print("\n✅ FAISS index elemzés kész")
else:
    print("❌ Nincs FAISS index az elemzéshez")

## 7. Következtetések

Az embedding kiértékelés összefoglalása.

In [ ]:
print("=== FAISS EMBEDDING ELEMZÉS ÖSSZEFOGLALÓ ===")
print("\n✅ Sikeresen elemezve:")
if index is not None:
    print(f"   📊 FAISS index: {index.ntotal} vektor, {index.d} dimenzió")
if chunk_id_map is not None:
    print(f"   🗺️ Chunk ID mapping: {len(chunk_id_map)} bejegyzés")
if df is not None:
    print(f"   📄 Chunk adatok: {len(df)} chunk betöltve")
    if 'embedding' in df.columns:
        valid_count = df['embedding'].notna().sum()
        print(f"   🧠 Embeddingek: {valid_count}/{len(df)} érvényes")

print("\n📋 Agents.md specifikáció ellenőrzés:")
if index is not None and index.d == config.EMBEDDING_DIMENSION:
    print("   ✅ Embedding dimenzió helyes")
else:
    print("   ❌ Embedding dimenzió eltérés")
    
# L2 normalizálás ellenőrzése
if df is not None and 'embedding' in df.columns:
    valid_embeddings = df.dropna(subset=['embedding'])
    if len(valid_embeddings) > 0:
        norms = valid_embeddings['embedding'].apply(lambda x: np.linalg.norm(x))
        normalized_count = norms.apply(lambda x: abs(x - 1.0) < 0.01).sum()
        if normalized_count >= len(valid_embeddings) * 0.95:  # 95% küszöb
            print("   ✅ L2 normalizálás megfelelő")
        else:
            print(f"   ⚠️ L2 normalizálás hiányos: {normalized_count}/{len(valid_embeddings)} ({100*normalized_count/len(valid_embeddings):.1f}%)")

print("\n💡 Ajánlások:")
if df is not None and 'embedding' in df.columns:
    missing = df['embedding'].isna().sum()
    if missing > 0:
        print(f"   🔄 Hiányzó embeddingek újragenerálása: {missing} chunk")
if index is None:
    print("   🚀 FAISS index generálása szükséges: gemma_embedding_runpod.ipynb")

print("\n🎯 Elemzés kész - a retrieval rendszer használatra kész!")